In [1]:
import requests
import time
import csv
import os
import pandas as pd

## Config (API_KEY 입력 필요함)

In [ ]:
# 설정
API_KEY = ""
HEADERS = {"X-Riot-Token": API_KEY}

# 협곡 솔랭만 검색
TIERS = ["IRON", "BRONZE", "SILVER", "GOLD", "PLATINUM", "EMERALD", "DIAMOND"]
DIVISIONS = ["I", "II", "III", "IV"]
QUEUE = "RANKED_SOLO_5x5"

# 플랫폼 & 호스트
platform_host = {
    "kr":"kr.api.riotgames.com",
    "br1":"br1.api.riotgames.com",
	"eun1":"eun1.api.riotgames.com",
	"euw1":"euw1.api.riotgames.com",
	"jp1":"jp1.api.riotgames.com",
	"la1":"la1.api.riotgames.com",
	"la2":"la2.api.riotgames.com",
    "me1":"me1.api.riotgames.com",
	"na1":"na1.api.riotgames.com",
	"oc1":"oc1.api.riotgames.com",
	"ru":"ru.api.riotgames.com",
	"sg2":"sg2.api.riotgames.com",
    "tr1":"tr1.api.riotgames.com",
	"tw2":"tw2.api.riotgames.com",
	"vn2":"vn2.api.riotgames.com"
}

## 메인

- 데이터 저장 & 수집 함수 (아이언 ~ 다이아)
- 티어, 디비젼, puuid만 저장함
- 다른 정보도 필요하면 수정해야함

In [3]:
def save_to_csv(data_list, filename):
    """수집된 PUUID 리스트를 CSV 파일에 이어씁니다."""
    if not data_list:
        return
        
    file_exists = os.path.isfile(filename)
    
    with open(filename, mode='a', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        # 파일이 처음 생성될 때만 헤더 작성
        if not file_exists:
            writer.writerow(['tier', 'division', 'puuid', 'leagueId', 'leaguePoints', 'wins', 'losses', 'hotStreak', 'veteran', 'freshBlood', 'inactive'])
        
        for row in data_list:
            writer.writerow(row)
    print(f"  -> {len(data_list)}개 데이터 저장 완료.")

def collect_all_data(platform):
    """모든 티어, 디비전을 순회하는 메인 함수.
    platform은 platform_host의 key 입력"""
    
    BASE_URL = f"https://{platform}.api.riotgames.com"
    OUTPUT_FILE = f"{platform}_puuids_iron_diamond.csv"
    if os.path.exists(OUTPUT_FILE):
        os.remove(OUTPUT_FILE)
    total_count = 0
    
    for tier in TIERS:
        for division in DIVISIONS:
            print(f"\n===== 시작: {tier} {division} =====")
            page = 1
            
            while True:
                try:
                    # API req
                    url = f"{BASE_URL}/lol/league/v4/entries/{QUEUE}/{tier}/{division}?page={page}"
                    response = requests.get(url, headers=HEADERS)
                    
                    # 에러 처리
                    if response.status_code != 200:
                        print(f"Warning: {response.status_code} 에러 발생. 1초 대기 후 리트.")
                        time.sleep(1)
                        continue
                    
                    entries = response.json()
                    
                    # 더 이상 데이터가 없으면 다음 Division으로 넘어감
                    if not entries:
                        print(f"  -> {page}페이지 결과 없음. {tier} {division} 종료.")
                        break
                    
                    # 데이터 파싱
                    current_batch = []
                    for entry in entries:
                        # name = entry.get('summonerName', 'Unknown')
                        puuid = entry.get('puuid')
                        leagueid = entry.get('leagueId')
                        leaguepoints = entry.get('leaguePoints')
                        wins = entry.get('wins')
                        losses = entry.get('losses')
                        hotstreak = entry.get('hotStreak')
                        veteran = entry.get('veteran')
                        freshblood = entry.get('freshBlood')
                        inactive = entry.get('inactive')
                        
                        if puuid:
                            current_batch.append([tier, division, puuid, leagueid, leaguepoints, wins, losses, hotstreak, veteran, freshblood, inactive])
                    
                    # 파일 저장 (batch 단위)
                    save_to_csv(current_batch, OUTPUT_FILE)
                    total_count += len(current_batch)
                    
                    print(f"  [{tier} {division}] {page}페이지 수집 완료 ({len(current_batch)}명)")
                    
                    # 다음 페이지로
                    page += 1
                    
                    # Rate Limit 방지 (개발자 키: 1초당 20회 / 2분당 100회)
                    # 안전하게 1.2초 대기. 120초 100회 맞추려고
                    time.sleep(1.2)
                    
                except Exception as e:
                    print(f"오류 발생: {e}")
                    time.sleep(1)
                    # 오류 발생 시 continue
                    continue

    print(f"\n{platform}서버. {total_count}명의 PUUID가 저장되었습니다.")

- 데이터 저장 & 수집 함수 (마, 그마, 챌)

In [4]:
def collect_high_tier_data(platform):
    """
    마스터, 그랜드마스터, 챌린저 데이터를 모두 수집, 하나의 CSV에 저장
    """
    LEAGUES = ['masterleagues','grandmasterleagues','challengerleagues']
    OUTPUT_FILE = f"{platform}_puuids_high_tiers.csv"
    
    # 반복문 시작 전에 파일 open(mode='w'로 초기화)
    with open(OUTPUT_FILE, mode='w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        
        # 헤더
        writer.writerow(['tier', 'puuid', 'leaguePoints', 'wins', 'losses', 'hotStreak', 'veteran', 'freshBlood', 'inactive'])
        
        # 파일이 열린 상태로 순회
        for league in LEAGUES:
            queue = "RANKED_SOLO_5x5"
            BASE_URL = f"https://{platform}.api.riotgames.com"
            
            url = f"{BASE_URL}/lol/league/v4/{league}/by-queue/{queue}"
            
            print(f"--- {league} 데이터 요청 중... ---")
            
            try:
                response = requests.get(url, headers=HEADERS)
                
                if response.status_code != 200:
                    print(f"  -> API 호출 실패! 코드: {response.status_code}")
                    continue
    
                data = response.json()
                
                tier = data.get('tier')
                entries = data.get('entries', [])
                
                print(f"  -> {tier} 응답 완료. {len(entries)}명 저장 시작.")
                
                # 열려있는 파일(f)에 데이터를 이어 씀
                count = 0
                for entry in entries:
                    """여기는 디비젼 구분 없음"""
                    puuid = entry.get('puuid')
                    leaguepoints = entry.get('leaguePoints')
                    wins = entry.get('wins')
                    losses = entry.get('losses')
                    hotstreak = entry.get('hotStreak')
                    veteran = entry.get('veteran')
                    freshblood = entry.get('freshBlood')
                    inactive = entry.get('inactive')
                    # PUUID가 있는 경우에만 저장
                    if puuid: 
                        writer.writerow([tier, puuid, leaguepoints, wins, losses, hotstreak, veteran, freshblood, inactive])
                        count += 1
                
                print(f"  -> {count}명 저장 완료.")
                
                # 안전하게 다음 요청 전 대기
                time.sleep(1.2)

            except Exception as e:
                print(f"오류 발생 ({league}): {e}")

    print(f"\n작업 완료. {platform}서버. 파일명: {OUTPUT_FILE}")

## EXECUTE

In [5]:
platforms = ['me1']  # [k.lower() for k in platform_host.keys()]

In [6]:
for platform in platforms:
    collect_all_data(platform)
    df = pd.read_csv(f'{platform}_puuids_iron_diamond.csv')
    df = df.drop_duplicates(subset=['puuid'])
    df.to_csv(f'{platform}_puuids_iron_diamond.csv', index=False)


===== 시작: IRON I =====
  -> 205개 데이터 저장 완료.
  [IRON I] 1페이지 수집 완료 (205명)
  -> 64개 데이터 저장 완료.
  [IRON I] 2페이지 수집 완료 (64명)
  -> 3페이지 결과 없음. IRON I 종료.

===== 시작: IRON II =====
  -> 89개 데이터 저장 완료.
  [IRON II] 1페이지 수집 완료 (89명)
  -> 2페이지 결과 없음. IRON II 종료.

===== 시작: IRON III =====
  -> 22개 데이터 저장 완료.
  [IRON III] 1페이지 수집 완료 (22명)
  -> 2페이지 결과 없음. IRON III 종료.

===== 시작: IRON IV =====
  -> 9개 데이터 저장 완료.
  [IRON IV] 1페이지 수집 완료 (9명)
  -> 2페이지 결과 없음. IRON IV 종료.

===== 시작: BRONZE I =====
  -> 205개 데이터 저장 완료.
  [BRONZE I] 1페이지 수집 완료 (205명)
  -> 205개 데이터 저장 완료.
  [BRONZE I] 2페이지 수집 완료 (205명)
  -> 205개 데이터 저장 완료.
  [BRONZE I] 3페이지 수집 완료 (205명)
  -> 205개 데이터 저장 완료.
  [BRONZE I] 4페이지 수집 완료 (205명)
  -> 139개 데이터 저장 완료.
  [BRONZE I] 5페이지 수집 완료 (139명)
  -> 6페이지 결과 없음. BRONZE I 종료.

===== 시작: BRONZE II =====
  -> 205개 데이터 저장 완료.
  [BRONZE II] 1페이지 수집 완료 (205명)
  -> 205개 데이터 저장 완료.
  [BRONZE II] 2페이지 수집 완료 (205명)
  -> 205개 데이터 저장 완료.
  [BRONZE II] 3페이지 수집 완료 (205명)
  -> 205개 데이터 저장 완료.
  [BRONZE II] 4페이

In [7]:
for platform in platforms:
    collect_high_tier_data(platform)
    df = pd.read_csv(f'{platform}_puuids_high_tiers.csv')
    df = df.drop_duplicates(subset=['puuid'])
    df.to_csv(f'{platform}_puuids_high_tiers.csv', index=False)

--- masterleagues 데이터 요청 중... ---
  -> MASTER 응답 완료. 127명 저장 시작.
  -> 127명 저장 완료.
--- grandmasterleagues 데이터 요청 중... ---
  -> GRANDMASTER 응답 완료. 67명 저장 시작.
  -> 67명 저장 완료.
--- challengerleagues 데이터 요청 중... ---
  -> CHALLENGER 응답 완료. 50명 저장 시작.
  -> 50명 저장 완료.

작업 완료. me1서버. 파일명: me1_puuids_high_tiers.csv
